In [1]:
import keras
import numpy as np
np.random.seed(1337)
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

num_classes = 10

# dimension
rows, columns = 28, 28

# Load MNIST data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Reshape data as Keras's required format
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, rows, columns)
    X_test = X_test.reshape(X_test.shape[0], 1, rows, columns)
    input_shape = (1, rows, columns)
else:
    X_train = X_train.reshape(X_train.shape[0], rows, columns, 1)
    X_test = X_test.reshape(X_test.shape[0], rows, columns, 1)
    input_shape = (rows, columns, 1)

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

print('X_train shape:', X_train.shape)
print('Train samples:', X_train.shape[0])
print('Test samples:', X_test.shape[0])

# One hot encoding label Y (convert class vectors to binary class matrices)
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

# # Define the model
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(36, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Using Theano backend.


X_train shape: (60000, 28, 28, 1)
Train samples: 60000
Test samples: 10000


In [2]:
model.load_weights('LeNET_model_for_MNIST_padding_same_20200523.h5')
print("Loaded model from disk")

# model.load_model('mnist_model_padding_same_20200331.h5')

Loaded model from disk


In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 36)        5220      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               211800    
_________________________________________________________________
dense_2 (Dense)              (None, 84)               

In [4]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Test loss: 0.040889969301866794
Test accuracy: 0.9904000163078308


In [5]:
# L1_C1 conv2d_1 (Conv2D)            (None, 28, 28, 16)        160    DONE  
# _________________________________________________________________
# L2_P1 max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
# _________________________________________________________________
# L3_C2 conv2d_2 (Conv2D)            (None, 14, 14, 36)        5220   DONE
# _________________________________________________________________
# L4_P2 max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
# _________________________________________________________________
# L5_F1 flatten_1 (Flatten)          (None, 1764)              0         
# _________________________________________________________________
# L6_D1 dense_1 (Dense)              (None, 120)               211800    
# _________________________________________________________________
# L7_D2 dense_2 (Dense)              (None, 84)                10164     
# _________________________________________________________________
# L8    dropout_1 (Dropout)          (None, 84)                0         
# _________________________________________________________________
# L9_D3 dense_3 (Dense)              (None, 10)                850 

In [6]:
def float_to_hexadecimal(number):
    intBits=4
    decBits=4
    if decBits == 0:
        mx = pow(2,intBits-1) - 1 # maximum number
    else:
        mx = pow(2,intBits-1) - pow(2,-1*decBits) # maximum number
    mn = -1*pow(2,intBits-1) # minimum number
    if number > mx:
        print ("number:" + str(number) + " has been truncated to: " + str(mx))
        number = mx
    elif number < mn:
        print ("number:" + str(number) + " has been truncated to: " + str(mn))
        number = mn
    n = []
    m = 0
    if number < 0:
        n.append(1)
        m = -1*pow(2,intBits-1)
    else:
        n.append(0)
        m = 0
    for i in reversed(range(intBits-1)):
        m1 = m + pow(2,i)
        if number < m1:
            n.append(0)
        else:
            n.append(1)
            m = m1
    for i in range(1,decBits+1):
        m1 = m + pow(2,-1*i)
        if number < m1:
            n.append(0)
        else:
            n.append(1)
            m = m1
    return hex(int("".join(str(i) for i in n),2))[2:].zfill(2)

print('You have successfully defined the function!')

You have successfully defined the function!


In [7]:
print('L1_C1')
L=1  #layer
R=28 #input width
C=28 #input height
M=16 #output depth
N=1  #input depth
K=3  #kernel size
S=1  #stride
IR=(R-1)*S+K #30
IC=(C-1)*S+K #30
L1_C1_w = open('../dat_grad/L1_C1_w.dat','w')
L1_C1_b = open('../dat_grad/L1_C1_b.dat','w')
for mm in range(M):
    for nn in range(N):
        for ii in range(K):
            for jj in range(K):
                L1_C1_w.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[0][jj][ii][nn][mm]))+'\n')
    L1_C1_b.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[1][mm]))+'\n')
L1_C1_w.close()
L1_C1_b.close()
print('You have successfully saved L1_C1 weight and bias')
print('w: '+str(model.layers[L-1].get_weights()[0].shape))
print('b: '+str(model.layers[L-1].get_weights()[1].shape))

L1_C1
You have successfully saved L1_C1 weight and bias
w: (3, 3, 1, 16)
b: (16,)


In [6]:
print('L1_C1')

for test_num in range(100):
    L1_C1_i = open('../dat_grad/L1_C1_i/L1_C1_i_'+str("{0:02d}".format(test_num))+'.dat','w')
    for x in X_test[test_num]:
        for y in x:
            L1_C1_i.write(str(float_to_hexadecimal(y[0]))+'\n')
    L1_C1_i.close()
print('You have successfully saved L1_C1 100 input (28x28) without padding test cases')

for test_num in range(100):
    L1_C1_i_pad = open('../dat_grad/L1_C1_i_pad/L1_C1_i_pad_'+str("{0:02d}".format(test_num))+'.dat','w')
    for i in range(30):
        L1_C1_i_pad.write('00'+'\n')
    for x in X_test[test_num]:
        L1_C1_i_pad.write('00'+'\n')
        for y in x:
            L1_C1_i_pad.write(str(float_to_hexadecimal(y[0]))+'\n')
        L1_C1_i_pad.write('00'+'\n')
    for i in range(30):
        L1_C1_i_pad.write('00'+'\n')    
    L1_C1_i_pad.close()
print('You have successfully saved L1_C1 100 input (30x30) with padding test cases')

L1_C1
You have successfully saved L1_C1 100 input (28x28) without padding test cases
You have successfully saved L1_C1 100 input (30x30) with padding test cases


In [9]:
print('L3_C2')
L=3  #layer
R=14 #input width
C=14 #input height
M=36 #output depth
N=16 #input depth
K=3  #kernel size
S=1  #stride
IR=(R-1)*S+K
IC=(C-1)*S+K

L3_C2_w = open('../dat_grad/L3_C2_w.dat','w')
L3_C2_b = open('../dat_grad/L3_C2_b.dat','w')
for mm in range(M):
    for nn in range(N):
        for ii in range(K):
            for jj in range(K):
                L3_C2_w.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[0][jj][ii][nn][mm]))+'\n')
    L3_C2_b.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[1][mm]))+'\n')
L3_C2_w.close()
L3_C2_b.close()
print('You have successfully saved L3_C2 weight and bias')

print('w: '+str(model.layers[L-1].get_weights()[0].shape))
print('b: '+str(model.layers[L-1].get_weights()[1].shape))

L3_C2
You have successfully saved L3_C2 weight and bias
w: (3, 3, 16, 36)
b: (36,)


In [ ]:
print('L6_D1 dense_1 (Dense)              (None, 120)               211800')
L=6  #layer
R=1 #input width
C=1 #input height
M=120 #output depth
N=1764 #input depth
K=0  #kernel size
S=0  #stride
IR=(R-1)*S+K
IC=(C-1)*S+K

L6_D1_w = open('../dat_grad/L6_D1_w.dat','w')
L6_D1_b = open('../dat_grad/L6_D1_b.dat','w')
for mm in range(M):
    for nn in range(N):
        L6_D1_w.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[0][nn][mm]))+'\n')
    L6_D1_b.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[1][mm]))+'\n')
L6_D1_w.close()
L6_D1_b.close()
print('You have successfully saved L6_D1 weight and bias')
print('w: '+str(model.layers[L-1].get_weights()[0].shape))
print('b: '+str(model.layers[L-1].get_weights()[1].shape))

In [ ]:
print('L7_D2 dense_2 (Dense)              (None, 84)                10164')
L=7  #layer
R=1 #input width
C=1 #input height
M=84 #output depth
N=120 #input depth
K=0  #kernel size
S=0  #stride
IR=(R-1)*S+K
IC=(C-1)*S+K

L7_D2_w = open('../dat_grad/L7_D2_w.dat','w')
L7_D2_b = open('../dat_grad/L7_D2_b.dat','w')
for mm in range(M):
    for nn in range(N):
        L7_D2_w.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[0][nn][mm]))+'\n')
    L7_D2_b.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[1][mm]))+'\n')
L7_D2_w.close()
L7_D2_b.close()
print('You have successfully saved L6_D1 weight and bias')
print('w: '+str(model.layers[L-1].get_weights()[0].shape))
print('b: '+str(model.layers[L-1].get_weights()[1].shape))

In [ ]:
print('L9_D3 dense_3 (Dense)              (None, 10)                850')
L=9  #layer
R=1 #input width
C=1 #input height
M=10 #output depth
N=84 #input depth
K=0  #kernel size
S=0  #stride
IR=(R-1)*S+K
IC=(C-1)*S+K

L9_D3_w = open('../dat_grad/L9_D3_w.dat','w')
L9_D3_b = open('../dat_grad/L9_D3_b.dat','w')
for mm in range(M):
    for nn in range(N):
        L9_D3_w.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[0][nn][mm]))+'\n')
    L9_D3_b.write(str(float_to_hexadecimal(model.layers[L-1].get_weights()[1][mm]))+'\n')
L9_D3_w.close()
L9_D3_b.close()
print('You have successfully saved L6_D1 weight and bias')
print('w: '+str(model.layers[L-1].get_weights()[0].shape))
print('b: '+str(model.layers[L-1].get_weights()[1].shape))